## Complete tree printing

In [14]:
import re

def parse_log_file(file_path):
    with open(file_path, 'r') as f:
        logs = []
        for line in f:
            # Match for function entry logs with '>' or '-->'
            match_push = re.match(r'^(--+>|\s*>)\s*call function (.+?) in (.+?):(\d+)', line)
            if match_push:
                function_name = match_push.group(2)
                file_path = match_push.group(3)
                line_number = int(match_push.group(4))
                logs.append((function_name, file_path, line_number, "push"))
                continue

            # Match for function exit logs with '<' or '<--'
            match_pop = re.match(r'^(<--+|\s*<)\s*exit function (.+?) in (.+?):(\d+)', line)
            if match_pop:
                function_name = match_pop.group(2)
                file_path = match_pop.group(3)
                line_number = int(match_pop.group(4))
                logs.append((function_name, file_path, line_number, "pop"))
                continue

    return logs

class CallStackNode:
    def __init__(self, function_name, file_path, line_number):
        self.function_name = function_name
        self.file_path = file_path
        self.line_number = line_number
        self.children = []
        self.parent = None

    def add_child(self, child_node):
        child_node.parent = self
        self.children.append(child_node)

    def __repr__(self, level=0, is_last_child=True, parent_last_childs=[]):
        indent = ''
        for parent_last in parent_last_childs:
            indent += '    ' if parent_last else '│   '

        branch = '└── ' if is_last_child else '├── '
        repr_str = f"{indent}{branch}{self.function_name}, {self.file_path}:{self.line_number}\n" # complete
        # repr_str = f"{indent}{branch}{self.function_name}\n" # simple without file path and line number

        for i, child in enumerate(self.children):
            repr_str += child.__repr__(level + 1, i == len(self.children) - 1, parent_last_childs + [is_last_child])
        return repr_str

class CallStackTree:
    def __init__(self):
        self.root = CallStackNode("root", "", 0)
        self.current_node = self.root

    def push(self, function_name, file_path, line_number):
        new_node = CallStackNode(function_name, file_path, line_number)
        self.current_node.add_child(new_node)
        self.current_node = new_node

    def pop(self):
        if self.current_node != self.root:
            self.current_node = self.current_node.parent

    def __repr__(self):
        return self.root.__repr__()

# Parse log file and construct call stack tree
log_file_path = "/root/vescale_prj/veScale/test/parallel/pipeline/instruction/logs-test_schedule-host_f78e8e970a17-pid_199624-py/tracing-test_schedule-20240829_071457.log"
logs = parse_log_file(log_file_path)

call_stack_tree = CallStackTree()

for log in logs:
    function_name, file_path, line_number, operation = log
    if operation == "push":
        call_stack_tree.push(function_name, file_path, line_number)
    elif operation == "pop":
        call_stack_tree.pop()

print(call_stack_tree)

└── root, :0
    ├── init_device_mesh, /root/vescale_prj/veScale/vescale/devicemesh_api/api.py:48
    │   └── init_device_mesh, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:594
    │       └── __init__, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:224
    │           ├── update_vescale_debug_mode_from_env, /root/vescale_prj/veScale/vescale/debug/debug_log.py:88
    │           ├── _get_or_create_default_group, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:313
    │           │   └── _get_device_handle, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:153
    │           ├── _get_device_handle, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:153
    │           ├── _get_current_device, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:281
    │           ├── _validate_mesh, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.py:339
    │           └── _init_process_groups, /root/vescale_prj/veScale/vescale/dtensor/device_mesh.

### Study plan

In [4]:
import re
import os
from collections import deque

def parse_log_file(file_path):
    with open(file_path, 'r') as f:
        logs = []
        for line in f:
            # Match for function entry logs with '>' or '-->'
            match_push = re.match(r'^(--+>|\s*>)\s*call function (.+?) in (.+?):(\d+)', line)
            if match_push:
                function_name = match_push.group(2)
                file_path = match_push.group(3)
                line_number = int(match_push.group(4))
                logs.append((function_name, file_path, line_number, "push"))
                continue

            # Match for function exit logs with '<' or '<--'
            match_pop = re.match(r'^(<--+|\s*<)\s*exit function (.+?) in (.+?):(\d+)', line)
            if match_pop:
                function_name = match_pop.group(2)
                file_path = match_pop.group(3)
                line_number = int(match_pop.group(4))
                logs.append((function_name, file_path, line_number, "pop"))
                continue

    return logs

class CallStackNode:
    def __init__(self, function_name, file_path, line_number):
        self.function_name = function_name
        self.file_path = file_path
        self.line_number = line_number
        self.children = []
        self.parent = None

    def add_child(self, child_node):
        child_node.parent = self
        self.children.append(child_node)

    def __repr__(self, level=0, is_last_child=True, parent_last_childs=[]):
        indent = ''
        for parent_last in parent_last_childs:
            indent += '    ' if parent_last else '│   '

        branch = '└── ' if is_last_child else '├── '
        repr_str = f"{indent}{branch}{self.function_name}, {self.file_path}:{self.line_number}\n" # complete

        for i, child in enumerate(self.children):
            repr_str += child.__repr__(level + 1, i == len(self.children) - 1, parent_last_childs + [is_last_child])
        return repr_str

class CallStackTree:
    def __init__(self):
        self.root = CallStackNode("root", "", 0)
        self.current_node = self.root

    def push(self, function_name, file_path, line_number):
        new_node = CallStackNode(function_name, file_path, line_number)
        self.current_node.add_child(new_node)
        self.current_node = new_node

    def pop(self):
        if self.current_node != self.root:
            self.current_node = self.current_node.parent

    def breadth_first_traversal(self):
        queue = deque([self.root])
        visited = set()
        traversal_order = []
        while queue:
            current_node = queue.popleft()
            node_id = (current_node.function_name, current_node.file_path, current_node.line_number)
            if node_id not in visited:
                visited.add(node_id)
                traversal_order.append(current_node)
                queue.extend(current_node.children)
        return traversal_order

    def depth_first_traversal(self):
        stack = [self.root]
        visited = set()
        traversal_order = []

        while stack:
            current_node = stack.pop()
            node_id = (current_node.function_name, current_node.file_path, current_node.line_number)

            if node_id not in visited:
                visited.add(node_id)
                traversal_order.append(current_node)

                # Extend stack with children in reverse order to preserve depth-first traversal
                stack.extend(reversed(current_node.children))

        return traversal_order

    def build_bfs_tree(self):
        bfs_order = self.breadth_first_traversal()
        bfs_tree_root = CallStackNode("BFS Root", "", 0)
        node_mapping = {bfs_order[0]: bfs_tree_root}  # Map original root to new BFS tree root

        for node in bfs_order[1:]:
            parent = node.parent
            if parent in node_mapping:
                new_node = CallStackNode(node.function_name, node.file_path, node.line_number)
                node_mapping[parent].add_child(new_node)
                node_mapping[node] = new_node

        return bfs_tree_root

    def __repr__(self):
        return self.root.__repr__()

# Parse log file and construct call stack tree
log_file_path = "/root/vescale_prj/veScale/examples/nanogpt_4D_finetune/logs/2024_0911_1938_40/logs-finetune_4D-host_d3cb3edeb6a9-pid_432289-py/tracing-finetune_4D-2024_0911_1938_40.log"
logs = parse_log_file(log_file_path)

call_stack_tree = CallStackTree()

for log in logs:
    function_name, file_path, line_number, operation = log
    if operation == "push":
        call_stack_tree.push(function_name, file_path, line_number)
    elif operation == "pop":
        call_stack_tree.pop()

# Ensure the logs directory exists
logs_dir = "logs"
if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)

# Save and print Breadth-First Traversal
bfs_log_file = os.path.join(logs_dir, "bfs_traversal.log")
traversal_order = call_stack_tree.breadth_first_traversal()

print("\nLearning Plan (Breadth-First, No Duplicates):")
with open(bfs_log_file, 'w') as f:
    f.write("Learning Plan (Breadth-First, No Duplicates):\n")
    for node in traversal_order:
        if node.function_name != "root":
            output = f"'{node.function_name}' in file '{node.file_path}', line {node.line_number}."
            print(output)
            f.write(output + "\n")

# Save and print Depth-First Traversal
dfs_log_file = os.path.join(logs_dir, "dfs_traversal.log")
traversal_order = call_stack_tree.depth_first_traversal()

print("\nLearning Plan (Depth-First, No Duplicates):")
with open(dfs_log_file, 'w') as f:
    f.write("Learning Plan (Depth-First, No Duplicates):\n")
    for node in traversal_order:
        if node.function_name != "root":
            output = f"'{node.function_name}' in file '{node.file_path}', line {node.line_number}."
            print(output)
            f.write(output + "\n")

################################################################################
# ### Save and print Tree Representation (all log lines, long time to process)
# tree_log_file = os.path.join(logs_dir, "call_stack_tree.log")
# with open(tree_log_file, 'w') as f:
#     tree_repr = repr(call_stack_tree)
#     f.write(tree_repr)
#     print(tree_repr)
################################################################################

print(f"Breadth-First traversal saved to {bfs_log_file}")
print(f"Depth-First traversal saved to {dfs_log_file}")
# print(f"Call stack tree representation saved to {tree_log_file}")

################################################################################

# BFS Summary
bfs_summary = f"\n{len(traversal_order)} function calls processed."
with open(bfs_log_file, 'a') as f:
    f.write(bfs_summary)
print(bfs_summary)

# Build BFS Tree from traversal
bfs_tree_root = call_stack_tree.build_bfs_tree()

# Save and print BFS Tree Representation
bfs_tree_log_file = os.path.join(logs_dir, "bfs_tree.log")
with open(bfs_tree_log_file, 'w') as f:
    bfs_tree_repr = bfs_tree_root.__repr__()
    f.write(bfs_tree_repr)
    print(bfs_tree_repr)

print(f"Breadth-First tree representation saved to {bfs_tree_log_file}")


Learning Plan (Breadth-First, No Duplicates):
'<listcomp>' in file '/root/vescale_prj/veScale/examples/nanogpt_4D_finetune/finetune_4D.py', line 633.
'<module>' in file '/root/vescale_prj/veScale/examples/nanogpt_4D_finetune/finetune_4D.py', line 1.
'<dictcomp>' in file '/root/vescale_prj/veScale/examples/nanogpt_4D_finetune/finetune_4D.py', line 635.
'main' in file '/root/vescale_prj/veScale/examples/nanogpt_4D_finetune/finetune_4D.py', line 234.
'__cleanup' in file '/root/vescale_prj/veScale/vescale/checkpoint/api/vescale_checkpointer.py', line 234.
'init_device_mesh' in file '/root/vescale_prj/veScale/vescale/devicemesh_api/api.py', line 48.
'get' in file '/root/vescale_prj/veScale/vescale/devicemesh_api/api.py', line 135.
'manual_seed' in file '/root/vescale_prj/veScale/vescale/dtensor/random.py', line 62.
'from_pretrained' in file '/root/vescale_prj/veScale/examples/nanogpt_4D_finetune/model.py', line 247.
'<genexpr>' in file '/root/vescale_prj/veScale/examples/nanogpt_4D_finetun